# **Data Acquisition and Organization**

In [0]:
# Import neccessary libraries
from datetime import datetime
import pandas as pd
import numpy as np

!pip install metpy
from metpy.io import parse_metar_to_dataframe
import glob

In [0]:
 from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Read in the data by month
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2001/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2002/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2003/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2004/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2005/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2006/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2007/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2008/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2009/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2010/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2011/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2012/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2013/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2014/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2015/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2016/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2017/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2018/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2019/64010KMSN*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2020/64010KMSN*.dat'

In [0]:
def parse_metar_file(file, wx_subset=True):
    """
    Parses METAR file from NCDC

    Input:
    --------
    file = Text file downloaded from NCDC

    wx_subset = Flag to determine whether or not to drop non-current weather obs (if True, only returns obs with observed weather)

    Output:
    --------
    df = Pandas dataframe filtered for times where current weather is not 'nan'
    """

    # Read in the file using Pandas
    df = pd.read_csv(file, header=None)

    # Pull the timestamp from the filename
    timestamp = datetime.strptime(file[-10:], '%Y%m.dat')

    # Iterrate over rows to parse METARS
    df_list = []
    for index, row in df.iterrows():
        try:
            df_list.append(parse_metar_to_dataframe(row.values[0][52:], year=timestamp.year, month=timestamp.month))
        except:
            print('Error with METAR: ', row.values[0][52:])
    #
    merged_df = pd.concat(df_list)

    # Drop datasets that do not include current weather
    merged_df = merged_df.dropna(subset=['current_wx1'])

    # Change the index to datetime
    merged_df.index = merged_df.date_time

    # Return the merged dataset sorted by datetime
    return merged_df.sort_index()

In [0]:
# Find all files in the data directory
files = sorted(glob.glob('*.dat'))

# Create a list to store the dataframes in
merged_datasets = []

In [0]:
# Loop through and parse the different datasets
for file in files:
    print(file[-10:])
    try:
        merged_datasets.append(parse_metar_file(file))
    except:
        print("Error with :", file)


In [0]:
# Return the cleaned dataset
#clean_df = (merged_datasets)
decoded_metar_master = pd.concat(merged_datasets)

## Save the file to memory
#clean_df.to_csv('decoded_metar_dataset.csv')
# Print out the resulting dataset
decoded_metar_master

In [0]:
# Create month column and add to the dataframe
time = pd.to_datetime(decoded_metar_master['date_time'])
decoded_metar_master['month'] = time.dt.month
decoded_metar_master

In [0]:
## Drop columns we don't plan on using or have too many NaNs
decoded_metar_master = decoded_metar_master.drop(['low_cloud_type',	'low_cloud_level',	'medium_cloud_type',	'medium_cloud_level',	'high_cloud_type',	'high_cloud_level',	'highest_cloud_type',	'highest_cloud_level',	'cloud_coverage', 'air_pressure_at_sea_level'], axis=1)


In [0]:
#convert current_wx columns to list and also check how many rows we have 
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))
current_wx2 = decoded_metar_master['current_wx2'].to_list()
print(len(current_wx2))
current_wx3 = decoded_metar_master['current_wx3'].to_list()
print(len(current_wx3))

#Check to make sure we have some frozen AND liquid weather reported
print(current_wx1[0:50])

In [0]:
# List all of the current weather identifiers to identify non-precipitation identifiers to be removed
print(decoded_metar_master.current_wx1.unique())

In [0]:
# Create non-precip variables list to replace with NaNs
non_precip = ['HZ', 'BR', 'FG', '-FZDZ', 'BL', 'TS', 'VCTS', 'MIFG', 'FZFG', '-SNDZ', 'VCFG', '+TS', '-TS', 'FZDZ', '-TSDZ']

In [0]:
# Replace non-precip variables with NaNs
decoded_metar_master.replace(non_precip, np.nan, inplace=True)
### Then drop current_wx2 and current_wx3 because they hold a lot of NaNs
decoded_metar_master = decoded_metar_master.drop(['current_wx2',	'current_wx3'], axis=1)
### Then drop all other rows with NaNs
decoded_metar_master = decoded_metar_master.dropna(axis=0)
### Check how many rows we are left with after cleaning up the dataframe
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))

In [0]:
### List of frozen and liquid precip options
fzn_list = ['-SNRA','SNRA','+SNRA','-SN','SN','+SN', '-TSSN', 'TSSN', '+TSSN']
liq_list = ['-RA','RA','+RA','-TS','TS','+TS','-TSRA','TSRA','+TSRA','-FZRA','FZRA','+FZRA']

In [0]:
### Set up a list to hold binary values (1 for frozen, 0 for liquid)
binary_list = []

### Loop over current_wx observations and determin if any are frozen precip. If so, set binary variable to 1. If not, set binary variable to 0.
for i in range(len(current_wx1)):
    if (current_wx1[i] in fzn_list) or (current_wx2[i] in fzn_list) or (current_wx3[i] in fzn_list):
        binary_list.append(1)
    else:
        binary_list.append(0)

### Fill a new column called 'fzn_or_liq' with the binary values.
decoded_metar_master['fzn_or_liq'] = binary_list

In [0]:
# Double check remaining parameters are correct and all precipitation
decoded_metar_master.current_wx1.unique()

In [0]:
# View final dataframe
decoded_metar_master

In [0]:
# Save the final data to a csv file, THIS IS IN THE REPOSITORY FOR EASY ACCESS FOR THE MODEL TO AVOID RUNNING THE ABOVE MORE THAN ONCE
decoded_metar_master.to_csv('/content/drive/My Drive/decoded_metar_FINAL.csv')

# **Basic Logistic Regression Model**

In [0]:
### Read all years data file
filename = '/content/drive/My Drive/decoded_metar_FINAL.csv' #Michael filepath
#filename = 'drive/My Drive/Colab Notebooks/ATMS-597/Project05/decoded_metar_FINAL.csv' #Kevin filepath
decoded_metar_master = pd.read_csv(filename,dtype={'current_wx1':str})

In [0]:
# View the dataframe
decoded_metar_master

In [0]:
### Split data 70/30 between training/testing
from sklearn.model_selection import train_test_split
#predictor_cols = ['air_temperature']
predictor_cols = ['air_temperature', 'wind_direction', 'wind_speed', 'dew_point_temperature', 'eastward_wind', 'northward_wind', 'month']
target_cols = ['fzn_or_liq']
X = decoded_metar_master[predictor_cols]
Y = decoded_metar_master[target_cols]
training_predictor, test_predictor, training_target, test_target = train_test_split(X, Y, test_size=0.3)

In [0]:
# Import needed sklearn modules
from sklearn.linear_model import LogisticRegression

In [0]:
# RUN THE TEST/TRAIN SPLIT CELL AND THIS CELL AGAIN WITH ITERATION FAILURE
# Initiate the model using the default parameters
logreg = LogisticRegression()

# Fit the model
#logreg.fit(training_predictor_array.reshape(-1,1), training_target['fzn_or_liq'])
logreg.fit(training_predictor, training_target['fzn_or_liq'])

# Predictions
ptype_predict=logreg.predict(test_predictor)
predsprob = logreg.predict_proba(test_predictor)

In [0]:
#print(len(predsprob))
prob0 = []
prob1 = []
for i in range(len(predsprob)):
    prob0.append(predsprob[i][0])
    prob1.append(predsprob[i][1])

In [0]:
# Check skill with confusion matrix
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(test_target['fzn_or_liq'], ptype_predict)
cnf_matrix

In [0]:
# Plot confusion matrix

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [0]:
# Other performance metrics

#Jaccard Index, want this to be close to one
from sklearn.metrics import jaccard_score
j_index = jaccard_score(y_true=test_target['fzn_or_liq'],y_pred=ptype_predict)
round(j_index,2)
print('j_index:',j_index)

# F1-score
from sklearn.metrics import f1_score
f1 = f1_score(test_target['fzn_or_liq'], ptype_predict)
print('f1 score',f1)

# Brier skill score
##from sklearn.metrics import brier_score_loss
##log_score = brier_score_loss((test_target['fzn_or_liq'].values).reshape(-1,1), predsprob[:][0])
##print('Brier:',log_score)
from sklearn.metrics import brier_score_loss
brier_score = brier_score_loss(test_target['fzn_or_liq'],prob1)
print('Brier:',brier_score)

# Precision score
from sklearn.metrics import precision_score
precision_score = precision_score(test_target['fzn_or_liq'], ptype_predict)
print('precision score:', precision_score)

In [0]:
### Get the climatology of fzn_or_liq
a = decoded_metar_master['fzn_or_liq']
counts = a.value_counts()
totalcounts = counts[0]+counts[1]
prcntliq = counts[0]/totalcounts
prcntfzn = counts[1]/totalcounts
print('Percent liquid reports:',prcntliq)
print('Percent frozen reports:',prcntfzn)

In [0]:
### Now make a new artificial probability lists filled with climatology values
#print(len(predsprob))
probCLIMO0 = []
probCLIMO1 = []
for i in range(len(predsprob)):
    probCLIMO0.append(prcntliq)
    probCLIMO1.append(prcntfzn)


### Get Brier score for this reference prediction
# Brier skill score
##from sklearn.metrics import brier_score_loss
##log_score = brier_score_loss((test_target['fzn_or_liq'].values).reshape(-1,1), predsprob[:][0])
##print('Brier:',log_score)
from sklearn.metrics import brier_score_loss
brier_score_reference = brier_score_loss(test_target['fzn_or_liq'],probCLIMO1)
print('Reference Brier:',brier_score)

In [0]:
### Calculate Brier Skill Score = (BSref-BSmodel)/BSref
brier_skill_score = (brier_score_reference-brier_score)/brier_score_reference
print('Brier Skill Score:',brier_skill_score)

In [0]:
# Validation ROC Curve

false_pos_rate, true_pos_rate, thresholds = roc_curve(test_target['fzn_or_liq'],prob1)

plt.plot([0, 1], [0, 1], linestyle='-.', color='g')
plt.plot(false_pos_rate, true_pos_rate)
plt.title('Validation ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# **Decision Tree Model**

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#change directory to the right spot
#import os 
#os.chdir('/content/gdrive/My Drive/ATMS_597_Project_5')

from datetime import datetime
import pandas as pd
!pip install metpy
from metpy.io import parse_metar_to_dataframe
import glob
import numpy as np
from sklearn import tree
from sklearn.metrics import roc_curve
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# Read data
filename= '/content/drive/My Drive/decoded_metar_FINAL.csv'
decoded_metar_dataset=pd.read_csv(filename,dtype={'current_wx1':str})
decoded_metar_dataset

##Rain/Liquid: RA, FZRA, TSRA =value 0  Snow/Freezing: SN, SNDZ=value 1)

In [0]:
where_snow_rain = [i for i,x in enumerate(decoded_metar_dataset['current_wx1'].to_list()) if ('RA' in x) | ('SN' in x)]
rain_snow = decoded_metar_dataset.iloc[where_snow_rain]
rain_snow.index = pd.to_datetime(rain_snow.index)
exclude_both = [i for i,x in enumerate(rain_snow['current_wx1'].to_list()) if 'SNRA' not in x]
rain_snow = rain_snow.iloc[exclude_both]
set(rain_snow['current_wx1'].to_list())

In [0]:
rain_snow['snow'] = 0
where_snow = [i for i,x in enumerate(rain_snow['current_wx1'].to_list()) if 'SN' in x]
rain_snow['snow'][where_snow] = 1

rain_snow['day_of_year'] = rain_snow.index.dayofyear

In [0]:
features = ['air_temperature', 'wind_speed', 'wind_direction',  
            'dew_point_temperature', 
            'altimeter', 'eastward_wind', 
            'northward_wind', 'day_of_year',
            'snow']

rain_snow_sub = rain_snow.dropna(subset = features)
train_X, valid_X, train_y, valid_y = train_test_split(rain_snow_sub[features[:-1]], rain_snow_sub['snow'], 
                                                  test_size = 0.3, random_state = 50)

In [0]:
dtree    = tree.DecisionTreeClassifier()
model  = dtree.fit(train_X, train_y)
prediction = model.predict(valid_X)

valid_prob = model.predict_proba(valid_X)
train_prob = model.predict_proba(train_X)

In [0]:
def get_climo(y, shape):
    climo = (np.size((y == 1).values.nonzero()))/float(np.size(y))
    prob  = np.zeros((shape))
    prob[:,0] = 1-climo
    prob[:,1] = climo

    return prob

In [0]:
train_climo_prob = get_climo(train_y, train_prob.shape)
valid_climo_prob = get_climo(valid_y, valid_prob.shape)
bsscore_train = 1 - (brier_score_loss(train_y, train_prob[:,1])/brier_score_loss(train_y, train_climo_prob[:,1]))
bsscore_valid = 1 - (brier_score_loss(valid_y, valid_prob[:,1])/brier_score_loss(valid_y, valid_climo_prob[:,1]))

In [0]:
print('Training Brier skill score: ' + str(bsscore_train))
print('Validation Brier skill score: ' + str(bsscore_valid))

In [0]:
# Validation ROC Curve

false_pos_rate, true_pos_rate, thresholds = roc_curve(valid_y, valid_prob[:,1])

plt.plot([0, 1], [0, 1], linestyle='-.', color='g')
plt.plot(false_pos_rate, true_pos_rate)
plt.title('Validation ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()